In [ ]:
!pip install skyfield pgmpy numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.2/336.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 11.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
from skyfield.api import load, EarthSatellite
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [ ]:
# load NORAD TLE data
def load_tle(tle_file_path):
  satellites = {}
  with open(tle_file_path, 'r') as file:
    lines = file.readlines()
    for i in range(0, len(lines), 3):
      name = lines[i].strip()
      line1 = lines[i + 1].strip()
      line2 = lines[i + 2].strip()
      satellite = EarthSatellite(line1, line2, name)
      satellites[name] = satellite
  return satellites

In [ ]:
tle_file_path = 'active.txt'
satellites = load_tle(tle_file_path)

In [ ]:
# calculate positional uncertainty and relative velocity
def calculate_uncertainty_and_velocity(satellite1, satellite2, time):
  # get positions at specified time
  position1 = satellite1.at(time).position.km
  position2 = satellite2.at(time).position.km

  # calculate Euclidean distance
  distance = np.linalg.norm(position1 - position2)

  # calculate relative velocity
  velocity1 = satellite1.at(time).velocity.km_per_s
  velocity2 = satellite2.at(time).velocity.km_per_s
  relative_velocity = np.linalg.norm(velocity1 - velocity2)

  return distance, relative_velocity

In [ ]:
ts = load.timescale()
time = ts.now()

In [ ]:
# example w/ two satellites from data
satellite_names = list(satellites.keys())
satellite1_name = satellites[satellite_names[0]]
satellite2_name = satellites[satellite_names[1]]

distance, relative_velocity = calculate_uncertainty_and_velocity(satellite1_name, satellite2_name, time)
print(f"Positional Distance (Uncertainty): {distance:.2f} km")
print(f"Relative Velocity: {relative_velocity:.2f} km/s")

Positional Distance (Uncertainty): 14161.99 km
Relative Velocity: 13.97 km/s


In [ ]:
# bayesian network model
model = BayesianNetwork([
    ("Debris_Position", "Collision_Risk"),
    ("Satellite_Position", "Collision_Risk"),
    ("Relative_Velocity", "Collision_Risk")
])

# define CPDs
cpd_debris_position = TabularCPD(
    variable="Debris_Position",
    variable_card=2,  # [low uncertainty, high uncertainty]
    values=[[0.8], [0.2]] # example probabilities
)
cpd_satellite_position = TabularCPD(
    variable="Satellite_Position",
    variable_card=2,  # [low uncertainty, high uncertainty]
    values=[[0.85], [0.15]]  # example probabilities
)
cpd_relative_velocity = TabularCPD(
    variable="Relative_Velocity",
    variable_card=2,  # [low velocity, High velocity]
    values=[[0.9], [0.1]]  # example probabilities
)
cpd_collision_risk = TabularCPD(
    variable="Collision_Risk",
    variable_card=2,
    evidence=["Debris_Position", "Satellite_Position", "Relative_Velocity"],
    evidence_card=[2, 2, 2],
    values=[
        [0.99, 0.95, 0.90, 0.85, 0.96, 0.90, 0.80, 0.70],  # no collision
        [0.01, 0.05, 0.10, 0.15, 0.04, 0.10, 0.20, 0.30]   # collision
    ]
)

model.add_cpds(cpd_debris_position, cpd_satellite_position, cpd_relative_velocity, cpd_collision_risk)
model.check_model()

True

In [ ]:
# perform inference
inference = VariableElimination(model)

In [ ]:
# map distance and velocity to network inputs
def map_to_bayesian_inputs(distance, relative_velocity):
    debris_uncertainty = 1 if distance > 100 else 0
    satellite_uncertainty = 1 if distance > 50 else 0
    velocity_category = 1 if relative_velocity > 2.0 else 0
    return debris_uncertainty, satellite_uncertainty, velocity_category

debris_uncertainty, satellite_uncertainty, velocity_category = map_to_bayesian_inputs(distance, relative_velocity)

# Query collision risk
query_result = inference.query(
    variables=["Collision_Risk"],
    evidence={
        "Debris_Position": debris_uncertainty,
        "Satellite_Position": satellite_uncertainty,
        "Relative_Velocity": velocity_category
    }
)

In [ ]:
print("\nCollision Probability Distribution:")
print(query_result)


Collision Probability Distribution:
+-------------------+-----------------------+
| Collision_Risk    |   phi(Collision_Risk) |
+===================+=======================+
| Collision_Risk(0) |                0.7000 |
+-------------------+-----------------------+
| Collision_Risk(1) |                0.3000 |
+-------------------+-----------------------+
